In [1]:
# !pip uninstall -y transformers
# !mkdir temp && cp -r /kaggle/input/transformers/transformers-main temp/transformers && cd temp/transformers && python setup.py develop --no-deps

In [2]:
!pip show transformers

Name: transformers
Version: 4.30.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [3]:
import sys
sys.path.insert(0, "/kaggle/working/temp/transformers/src/")

In [4]:
import pandas as pd
import io
import torch
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, AutoConfig
import requests
from PIL import Image
from PIL import ImageEnhance
import re

import torch
import torchvision
import torch.utils.data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2
import numpy as np

import os
from tqdm import tqdm
import math

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
def display_deplot_output(deplot_output):
    '''
    The raw output of deplot
    TITLE | Rural population (%) long-run with 2050 projections<0x0A>(OWID) in Greece<0x0A>Years | Rural population<0x0A>1940 | 47.38<0x0A>1960 | 43.68<0x0A>1980 | 30.28<0x0A>...
    '''

    # x1,x2 <> y1 y2
    # x1 | y1 <0x0A> x2 | y2 <0x0A>
    # label xxxxyyy xyxyxyxy
    # <x_start><x_end> <y_start><y_end>

#     deplot_output = deplot_output.replace("<0x0A>", "\n").replace(" | ", "\t")
#     print(deplot_output)
#     second_a_index = [m.start() for m in re.finditer('\t', deplot_output)][1]
#     last_newline_index = deplot_output.rfind('\n', 0, second_a_index) 

#     title = deplot_output[:last_newline_index]
#     table = deplot_output#[last_newline_index+1:]

#     data = io.StringIO(table)
#     df = pd.read_csv(data, sep='\t', names=['x', 'y'])
#     if visualize:
#         display(df)
#     deplot_output = deplot_output.split("<x_end> <y_start>")
#     x_values=deplot_output[0].split("<x_start>")[1].split(";")
#     y_values=deplot_output[1].split("<y_end>")[0].split(";")
    if "<x_end><y_start>" in deplot_output:
        deplot_output = deplot_output.split("<x_end><y_start>")
    else:
        deplot_output = deplot_output.split("<x_end> <y_start>")
    x_values=deplot_output[0].split("<x_start>")[1].split(";")
    y_values=deplot_output[1].split("<y_end>")[0].split(";")
    
    return x_values, y_values
    

def deplot(path, model, processor, device, transforms=None):
    if transforms:
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms(image=image)['image']
    else:
        image = Image.open(path)
#     w, h = image.size
#     tosize = 500
#     image = image.resize((tosize,int(tosize/w*h)),Image.Resampling.BICUBIC)
    
#     image = cv2.imread(path)
    # 双边滤波
#     image = cv2.bilateralFilter(src=image, d=0, sigmaColor=30, sigmaSpace=15)
    
#     # 对比度增强
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     contrast = clahe.apply(gray)
#     image = cv2.cvtColor(contrast, cv2.COLOR_BGR2RGB)

#     # 修改图片尺寸大小
#     h,w,c = image.shape
#     tosize = 600
#     image = cv2.resize(image,(tosize,int(tosize/w*h)-5),interpolation=cv2.INTER_LANCZOS4)
    
    
    inputs = processor(images=image, 
                       font_path="../input/arial-font/arial.ttf",
                       text="Generate underlying data table of the figure below:", 
                       return_tensors="pt", 
                       add_special_tokens=True, 
                       max_patches=2048,
                       is_vqa=False)

    # Move inputs to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.cuda.amp.autocast():
        predictions = model.generate(**inputs, max_new_tokens=512)
#     predictions = model.generate(**inputs, max_new_tokens=512)
    return processor.decode(predictions[0], skip_special_tokens=True)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

deplot_weights_path = '/kaggle/input/matcha-base/matcha-base'
processor = Pix2StructProcessor.from_pretrained(deplot_weights_path)

model_path = '../input/checkpoints/best_matcha_xxyy_epoch_2_loss_0.6506.bin'  # 注意修改 max_patches 和 resize_token_embeddings
deplot_model_all = Pix2StructForConditionalGeneration.from_pretrained(deplot_weights_path).to(device)
# deplot_model_all.resize_token_embeddings(len(processor.tokenizer))
deplot_model_all.config.text_config.is_decoder = True
deplot_model_all.encoder.gradient_checkpointing_enable()
deplot_model_all.decoder.gradient_checkpointing_enable()
deplot_model_all.load_state_dict(torch.load(model_path))
# processor.is_vqa = False

<All keys matched successfully>

In [7]:
model_path = '../input/checkpoints/matcha_horizontal_bar_epoch_0_loss_0.1652.bin'  # 注意修改 max_patches 和 resize_token_embeddings
deplot_model_horizontal_bar = Pix2StructForConditionalGeneration.from_pretrained(deplot_weights_path).to(device)
deplot_model_horizontal_bar.config.text_config.is_decoder = True
deplot_model_horizontal_bar.encoder.gradient_checkpointing_enable()
deplot_model_horizontal_bar.decoder.gradient_checkpointing_enable()
deplot_model_horizontal_bar.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [8]:
model_path = '../input/checkpoints/matcha_xxyy_epoch_9_loss_0.7815.bin'  # 注意修改 max_patches 和 resize_token_embeddings
deplot_model_scatter = Pix2StructForConditionalGeneration.from_pretrained(deplot_weights_path).to(device)
deplot_model_scatter.config.text_config.is_decoder = True
deplot_model_scatter.encoder.gradient_checkpointing_enable()
deplot_model_scatter.decoder.gradient_checkpointing_enable()
deplot_model_scatter.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [9]:
augments = A.Compose([
        A.Resize(256, 256, p=1.0),
        A.Normalize((0, 0, 0), (1, 1, 1), p=1.0),
        ToTensorV2(),
])

In [10]:
def deplot_inference(image_path, chart_type):
    if chart_type == 'horizontal_bar':
        deplot_output=deplot(image_path, deplot_model_horizontal_bar, processor, device)
    elif chart_type == 'scatter' or chart_type == 'line' :
        deplot_output=deplot(image_path, deplot_model_scatter, processor, device)
    else:
#         deplot_output=deplot(image_path, deplot_model_all, processor, device)
        deplot_output=deplot(image_path, deplot_model_all, processor, device, augments)
#     print(image_path)
#     print(chart_type)
#     print(deplot_output)
    return display_deplot_output(deplot_output)

In [11]:
label_map = {'dot': 0, 'horizontal_bar' : 1, 'vertical_bar': 2, 'line': 3, 'scatter': 4}
label_idx_to_classname = {v: k for k, v in label_map.items()}
label_idx_to_classname

{0: 'dot', 1: 'horizontal_bar', 2: 'vertical_bar', 3: 'line', 4: 'scatter'}

In [12]:
# classification_model = torchvision.models.resnet50(pretrained=False)
# num_features = classification_model.fc.in_features
# classification_model.fc = nn.Linear(num_features, 5)

classification_model = torchvision.models.efficientnet_b2(pretrained=False)
classification_model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(1408, 5),
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classification_model = classification_model.to(device)
classification_model.eval()

# state_dict = torch.load("../input/checkpoints/Benetech_ResNet50_Acc_87.2093.pth")
state_dict = torch.load("../input/checkpoints/Benetech_Efficientnet_B2_Acc_88.1932.pth")
classification_model.load_state_dict(state_dict)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
# val_transforms = transforms.Compose([
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
#     transforms.ToTensor(),
# ])

val_transforms = A.Compose([
    A.Resize(288,288,p=1),
#     A.Resize(500,300,p=1),
#     A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ToTensorV2(),
])

def classification_inference(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img,(288,288))
#     img = img.astype(np.float32)/255.0
    img = val_transforms(image=img)['image']
    
#     print(img.shape)
#     inp = torch.stack([img.permute(1, 2, 0)]).cuda()

    inp = torch.stack([img]).cuda()
#     with torch.cuda.amp.autocast():
#         out = classification_model(inp).detach().cpu().numpy()
    out = classification_model(inp).detach().cpu().numpy()
    return label_idx_to_classname[np.argmax(out)]

In [14]:
IMAGE_FOLDER = "/kaggle/input/benetech-making-graphs-accessible/test/images"

images_name = os.listdir(IMAGE_FOLDER)
images_path = [os.path.join(IMAGE_FOLDER, i) for i in images_name if ".jpg" in i]
images_path

['/kaggle/input/benetech-making-graphs-accessible/test/images/000b92c3b098.jpg',
 '/kaggle/input/benetech-making-graphs-accessible/test/images/01b45b831589.jpg',
 '/kaggle/input/benetech-making-graphs-accessible/test/images/00f5404753cf.jpg',
 '/kaggle/input/benetech-making-graphs-accessible/test/images/00dcf883a459.jpg',
 '/kaggle/input/benetech-making-graphs-accessible/test/images/007a18eb4e09.jpg']

In [15]:
from datetime import datetime

chart_types_list = []
x_values_list = []
y_values_list = []

for image_path in tqdm(images_path):
    graph_type = classification_inference(image_path)
    chart_types_list.append(graph_type)

for image_path, chart_type in tqdm(zip(images_path,chart_types_list)):
    try:
        x_values, y_values = deplot_inference(image_path, chart_type)
    except Exception as e:
        print("Exception:", e ,", data:", image_path)
        x_values, y_values = ['0'], ['0']
    x_values_list.append(x_values)
    y_values_list.append(y_values)

100%|██████████| 5/5 [00:04<00:00,  1.19it/s]
5it [00:43,  8.79s/it]


In [16]:
all_ids = []
all_values = []
all_chart_types = []

for image_name, x_values, y_values, graph_type in tqdm(zip(images_name, x_values_list, y_values_list, chart_types_list)):
#     start = datetime.now()
    try:
        if len(graph_type) == 0:
            graph_type = "line"
            
        x_len = len(x_values) 
        y_len = len(y_values) 
        if x_len == 0:
            x_values.append("0")
        if y_len == 0:
            y_values.append("0")
        
#         print(x_values)
#         print(y_values)
        
    
        formatted_y_values = []
        formatted_x_values = []
        # check value type for each graph type
        if graph_type in ["horizontal_bar", "vertical_bar", "line", "dot", "scatter"]:
            # x is categorical, y is numerical
            
            for y in y_values:
                try:
                    v = float(y)
                    if math.isnan(v) or math.isinf(v):
                        formatted_y_values.append(0)
                    else:
                        formatted_y_values.append(v)
                except:
                    if len(y) == 0:
                        formatted_y_values.append(0)
                    else:
                        formatted_y_values.append(y)
                        
            for x in x_values:
                if len(x) == 0:
                    formatted_x_values.append(0)
                else:
                    formatted_x_values.append(x)
        
        elif graph_type in ["dot", "scatter"]:
            # for dot graph, x could be categorical as well as numerical, how to handle this?
            # just leave all the value be numerical for now
            
            for x in x_values:
                try:
                    v = float(x)
                    if math.isnan(v) or math.isinf(v):
                        formatted_x_values.append(0)
                    else:
                        formatted_x_values.append(v)
                except:
                    if len(x) == 0:
                        formatted_y_values.append(0)
                    else:
                        formatted_y_values.append(x)
            for y in y_values:
                if len(y) == 0:
                    formatted_y_values.append(0)
                else:
                    formatted_y_values.append(y)
                
        x_values = formatted_x_values
        y_values = formatted_y_values
                                   
        length = min(x_len, y_len)
        x_values = ";".join([str(v).strip() for v in x_values][:length])
        y_values = ";".join([str(v).strip() for v in y_values][:length])
    except Exception as e:
        print("Exception", e)
        graph_type = "line"
        x_values = "0;0"
        y_values = "0;0"
        
#     print()
#     print(x_values)
#     print(y_values)
#     print()

    image_id = image_name.split(".")[0]

    all_ids.append(image_id + "_x")
    all_values.append(x_values)
    all_ids.append(image_id + "_y")
    all_values.append(y_values)
    all_chart_types.extend([graph_type, graph_type])
    
#     end = datetime.now()
#     print(end-start)

5it [00:00, 10793.37it/s]


In [17]:
submission_df = pd.DataFrame({
    "id": all_ids,
    "data_series": all_values,
    "chart_type": all_chart_types
})

submission_df.to_csv("submission.csv", index=False)
submission_df

,id,data_series,chart_type
0,000b92c3b098_x,0;6;12;18;24,line
1,000b92c3b098_y,0.01375;-1.357142857142857;-2.666666666666666;...,line
2,01b45b831589_x,21-Fed;22-Fed;23-Fed;24-Fed;25-Fed;27-Fed;28-F...,vertical_bar
3,01b45b831589_y,88888.88888888889;149888.8888888889;171428.188...,vertical_bar
4,00f5404753cf_x,3.0303030303030305;3.9898989898989903;4.989898...,scatter
5,00f5404753cf_y,11.984422222222221;12.025675675675677;12.02567...,scatter
6,00dcf883a459_x,Group 1;Group 2,vertical_bar
7,00dcf883a459_y,3.811111111111111;8.61111111111111,vertical_bar
8,007a18eb4e09_x,0.0;0.4;0.8;1.2;1.6;2.0;2.4,line
9,007a18eb4e09_y,0.013275862068965517;0.013275862068965517;0.01...,line


In [18]:
!rm -rf temp